In [ ]:
import tempfile
from pathlib import Path

import os
import numpy as np
import pandas as pd
import seaborn as sb
from IPython.display import JSON, display, Markdown
from panoptes.utils.images import bayer
from panoptes.utils.images import fits as fits_utils
from panoptes.utils.serializers import from_json
from panoptes.utils.time import current_time

from panoptes.data.observations import ObservationPathInfo
from panoptes.pipeline.scripts import image as image_processing
from panoptes.pipeline.utils.plot import plot_bg_overlay, plot_raw_bg_overlay, plot_stellar_location

sb.set_theme()

os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = '1'

In [ ]:
current_time()

In [ ]:
# Input parameters
fits_path =  'https://storage.googleapis.com/panoptes-images/PAN019/42433a/20220120T050021/20220120T051632.fits.fz'
output_dir = tempfile.TemporaryDirectory().name
force_new = True
make_plots = False
save_extras = False

# JSON string of additional settings.
image_settings = '{}'

In [ ]:
# Parse the image settings.
image_settings = from_json(image_settings)

# Set up output directory and filenames.
output_dir = Path(output_dir)
output_dir.mkdir(parents=True, exist_ok=True)

# Override output dir with provided.
image_settings['output_dir'] = output_dir

# Get the settings object.
settings = image_processing.Settings(**image_settings)

# Prepend the output dir to the file objects.
for file_type, file_name in settings.files:
    setattr(settings.files, file_type, output_dir / file_name)

In [ ]:
path_info = ObservationPathInfo(path=fits_path)
image_uid = path_info.get_full_id(sep='/')

In [ ]:
Markdown(f'# Processing for {image_uid}')

### Get data

In [ ]:
raw_data, header = fits_utils.getdata(str(fits_path), header=True)

# Set some default header values.
header.setdefault('SEQID', path_info.sequence_id)
header.setdefault('IMAGEID', path_info.image_id)
header.setdefault('ELEV-OBS', 0)
header.setdefault('MOONFRAC', 0)
header.setdefault('MOONSEP', 0)

In [ ]:
settings.params.camera.image_height = raw_data.shape[0]
settings.params.camera.image_width = raw_data.shape[1]

### Mask bias and outliers

In [ ]:
# Subtract zero bias.
data = raw_data - settings.params.camera.zero_bias

# Mask saturated
data = np.ma.masked_greater_equal(data, settings.params.camera.saturation).astype('float32')

### Subtract RGB background

In [ ]:
# Get RGB background data.
rgb_background = bayer.get_rgb_background(data=data,
                                          return_separate=True,
                                          box_size=settings.params.background.box_size,
                                          filter_size=settings.params.background.filter_size,
                                          )

In [ ]:
combined_bg_data = list()
combined_bg_residual_data = list()
for color, bg in zip(bayer.RGB, rgb_background):
    color_data = np.ma.array(data=bg.background, mask=bg.mask)
    color_residual_data = np.ma.array(data=bg.background_rms, mask=bg.mask)

    combined_bg_data.append(color_data)
    combined_bg_residual_data.append(color_residual_data)

In [ ]:
# Combine the colors
bg_data = np.ma.array(combined_bg_data).filled(0).sum(0)
bg_residual_data = np.ma.array(combined_bg_residual_data).filled(0).sum(0)
reduced_data = data - bg_data

### Save FITS files

In [ ]:
# Save reduced data and background.
image_processing.save_fits(settings.files.reduced_filename, 
          dict(
              reduced=reduced_data.data.astype(np.float32)
          ),
          header,
          force_new=force_new)

if save_extras:
    image_processing.save_fits(settings.files.extras_filename,
              dict(
                  background=bg_data.astype(np.float32),
                  residual=bg_residual_data.astype(np.float32),
                  mask=reduced_data.mask.astype(np.uint8)
              ),
              header,
              force_new=force_new)

### Plate solve

In [ ]:
# Plate solve newly calibrated file.
wcs0 = image_processing.plate_solve(settings=settings)
wcs0

### Detect sources

In [ ]:
detected_sources = image_processing.detect_sources(wcs0, reduced_data, bg_data, bg_residual_data, settings=settings)

### Match detected sources to catalog

In [ ]:
matched_sources = image_processing.match_sources(detected_sources, wcs0, settings=settings)

### Get metadata from image

In [ ]:
metadata_headers = image_processing.get_metadata(header, matched_sources, settings=settings)

In [ ]:
# Write dataframe to csv.
matched_sources['time'] = pd.to_datetime(metadata_headers['image']['time'], utc=True)
matched_sources.set_index(['picid', 'time'], inplace=True)
matched_sources.to_parquet(settings.files.sources_filename)
print(f'Matched sources saved to {settings.files.sources_filename}')

In [ ]:
if settings.compress_fits:
    print(f'Compressing FITS files')
    fits_utils.fpack(str(settings.files.reduced_filename), overwrite=force_new)
    if save_extras:
        fits_utils.fpack(str(settings.files.extras_filename), overwrite=force_new)

## Plotting and visualizations

In [ ]:
if make_plots:
    fig = plot_raw_bg_overlay(reduced_data,
                        rgb_background=rgb_background[bayer.RGB.G], 
                        title=f'Background mesh over calibrated image {image_uid.replace("/", "_")}')
    
    fig.savefig(output_dir / f'bg-mesh-g-image.png')
    
    fig = plot_bg_overlay(bg_data,
                    rgb_background[bayer.RGB.G],
                    title=f'Background mesh over background {image_uid.replace("/", "_")}')
    
    fig.savefig(output_dir / f'bg-mesh-g-background.png')
    
    fig = plot_stellar_location(matched_sources, 
                          wcs=wcs0, 
                          title=f'{len(matched_sources)} detected sources for {image_uid}')
    
    fig.savefig(output_dir / f'stellar-locations.png')

    dist_columns = [
        'photutils_fwhm',
        'photutils_gini',
        'catalog_vmag',
        'catalog_sep',
    ]
    
    pg = sb.pairplot(matched_sources[dist_columns], kind='reg', plot_kws={
        'order': 2, 
        'line_kws':{'color':'red'}, 
        'scatter_kws': {'alpha': 0.1}}
                    )
    pg.fig.suptitle(f'Catalog and detected properties for {image_uid}', y=1.02)
    
    pg.fig.savefig(output_dir / f'{image_uid.replace("/", "_")}-pairplot.png')

In [ ]:
try:
    display(JSON(metadata_headers, expanded=True))
except Exception:
    from pprint import pprint
    pprint(metadata_headers)

In [ ]:
!jupyter --version

In [ ]:
import json
print(json.dumps(from_json(settings.model_dump_json()), indent=2))

In [ ]:
current_time()